In [1]:
# Charles Campbell
# April 20, 2025

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
# Necessary import for radio buttons
from dash import dcc
from dash import html
# Necessary import for Bar Graph
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### COMPLETE ON FIX ME #####
# changed animal_shelter and AnimalShelter to match my CRUD Python module file name and class name
from aac_crud_script import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# COMPLETE ON FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

# connection variables
username = "aacuser"
password = "user_password"
HOST = 'nv-desktop-services.apporto.com'
PORT = 34641
DB = 'AAC'
COL = 'animals'
# Instantiation to connect to the database
shelter = AnimalShelter(username,password,HOST,PORT,DB,COL)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare Dashboard - made by Charles Campbell')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Necessary code "html.A" to add the logo & make it clickable to 
    # Grazioso Salvare's website (snhu.edu)
    html.A([
        html.Center(html.Img(src='assets/logo.png', style={'width': '15%', 'height': 'auto'})),
    ], href='https://www.snhu.edu', target='_blank'),
    html.Center(html.B(html.H1('Grazioso Salvare Animal Dashboard'))),
    html.Center(html.B(html.H3('Developed by Charles Campbell'))),
    html.Hr(),
    # Radio Buttons used to filter/categorize certain results on the data table
    dcc.RadioItems(id='radio-options',options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Wilderness Rescue', 'value': 'wilderness'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'Reset Table', 'value': 'reset'},
        ], value='reset', inline=True
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # COMPLETE ON FIXME: Set up the features for your interactive data table to make
        # it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
     ),
     html.Br(),
     html.Hr(),
     html.Div(id='map-id',className='col s12 m6',),
     html.Div(id='graph-id')
])

# Campbell Addition: Callback to automatically ascending sort the table by id number
@app.callback(
    Output('datatable-id', 'sort_by'),
    Input('datatable-id', 'data')
)
def sort_initial_descending(data):
    if data:
        return [{'column_id': 'rec_num', 'direction': 'asc'}]
    return []

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Campbell Addition: Callback that allows user to click filtering buttons and adapt table results;
# buttons are in radio format, and only have to be pressed once for the table, geomap, and graph to be updated
@app.callback(
    Output('datatable-id', "data"),
    [Input('radio-options', 'value')])

def filtering_options(selected_value):
    # Displays standard full list of all animals in database
    df = pd.DataFrame.from_records(shelter.read({}))
    
    ### Queries based on specific filtering and functionality defined by client
    ### (Rescue Type and Preferred Dog Breeds Table referenced)
    
    # Water Rescue Preferred:
    # Breeds: Labrador Mix, Chesapeake Retriever, Newfoundland
    # Sex: Intact Female
    # Age in Weeks: greater than or equal to 26, and less than or equal to 156
    water = {'breed': { '$in': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
             'sex_upon_outcome': 'Intact Female',
             '$and':[{'age_upon_outcome_in_weeks':{'$gte': 26}}, {'age_upon_outcome_in_weeks':{"$lte":156}}]}
    
    # Wilderness Rescue Preferred:
    # Breeds: German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler
    # Sex: Intact Male
    # Age in Weeks: greater than or equal to 26, and less than or equal to 156
    wilderness = {'breed': { '$in': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                  'sex_upon_outcome': 'Intact Male',
                  '$and':[{'age_upon_outcome_in_weeks':{'$gte': 26}}, {'age_upon_outcome_in_weeks':{"$lte":156}}]}
    
    # Disaster Rescue Preferred:
    # Breeds: Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler
    # Sex: Intact Male
    # Age in Weeks: greater than or equal to 20, and less than or equal to 300
    disaster = {'breed': { '$in': ["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                'sex_upon_outcome': 'Intact Male',
                '$and':[{'age_upon_outcome_in_weeks':{'$gte': 20}}, {'age_upon_outcome_in_weeks':{"$lte":300}}]}
    
    # Upon click of the "water" button being triggered, app's visual data meets clients specifications
    if (selected_value == 'water'):
        df = pd.DataFrame.from_records(shelter.read(water))
    
    # Upon click of the "wilderness" button being triggered, app's visual data meets clients specifications
    elif (selected_value == 'wilderness'):
        df = pd.DataFrame.from_records(shelter.read(wilderness))
        
    # Upon click of the "disaster" button being triggered, app's visual data meets clients specifications
    elif (selected_value == 'disaster'):
        df = pd.DataFrame.from_records(shelter.read(disaster))
    
    # Upon click of the "reset" button being triggered, make the table revert back to normal (10,000 documents)
    elif (selected_value == 'reset'):
        df = pd.DataFrame.from_records(shelter.read({}))
    
    # Clean Mongo _id field from data table
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

def update_map(viewData, index):
#COMPLETE ON FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Convert the list to a row index
    if index is None:
        row = 0
    else:
        row = index[0]
    
    # Returns results near Austin TX: [30.26, -97.74]
    return [
        dl.Map(style={'width': '50%', 'height': '500px', 'float': 'right'},
        center=[30.26,-97.74], zoom=8,
        children=[
            # Marker with tool-tip & pop-up
            dl.TileLayer(id="base-layer-id"),
            # Column 13 & 14 define the grid-coordinates for the map
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
            children=[
                # Column 4 defines breed of animal
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    # Column 9 defines name of animal
                    html.H3("Animal Name:"),
                    html.H1(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Campbell Addition: Sunburst Pie Chart included here. Chart displays the breeds at the center, which when
# clicked, expand to show the names of the animals included in that breed. Additionally, upon hovering over
# the name of the animal, it will show the age in weeks of that animal to the viewer.
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    # Data pulled from the current view (changes based on filtered or not results)
    dff = pd.DataFrame.from_dict(viewData)
    
    # Return the results in a bar graph
    return [
        dcc.Graph(style={'width': '50%', 'height': '625px', 'float': 'left'},
            figure = px.sunburst(dff, path=['breed', 'name'], values='age_upon_outcome_in_weeks')
        )
    ]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:11277/


/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be remo